In [13]:
pwd

'/home/wsuser/work'

In [14]:
import keras
from keras.preprocessing.image import ImageDataGenerator


In [15]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [16]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0IvXDlMUbjNzuNEr1c_ifCjtrMd-zZrLgS-Yo-nGTFFq',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'emergingmethodsforearlydetectiono-donotdelete-pr-dcp0rw0te3koeo'
object_key = 'archive.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [17]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [18]:
##Applying ImageDataGenerator Functionality to trainset
x_train=train_datagen.flow_from_directory(r'./Dataset/Dataset/train_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 436 images belonging to 2 classes.


In [19]:
##Applying ImageDataGenerator Functionality to testset
x_test=test_datagen.flow_from_directory(r'./Dataset/Dataset/test_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 121 images belonging to 2 classes.


In [20]:
##Import model building libraries

#To Define linear initialisation import Sequential

from keras.models import Sequential

#To add layers import Dense

from keras.layers import Dense

#To create Convolution kernel import Convolution 2D

from keras.layers import Convolution2D

#import maxpooling layers

from keras.layers import MaxPooling2D

#import flatten Layer

from keras.layers import Flatten

import warnings
warnings.filterwarnings('ignore')

In [21]:
#Initializing the Model
model=Sequential()

In [22]:
##adding CNN layers
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

##adding maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

##adding flatten Layer
model.add(Flatten())

In [23]:
##add hidden layer

model.add(Dense(150,activation='relu'))

##add output layer

model.add(Dense(1,activation='sigmoid'))

In [24]:
#Configure the Learning Process

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [25]:
##Training the model
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 22s 2s/step - loss: 2.6359 - accuracy: 0.6697 - val_loss: 0.1864 - val_accuracy: 0.9339
Epoch 2/10
14/14 [==============================] - 21s 1s/step - loss: 0.2821 - accuracy: 0.8968 - val_loss: 0.1254 - val_accuracy: 0.9504
Epoch 3/10
14/14 [==============================] - 21s 1s/step - loss: 0.1955 - accuracy: 0.9106 - val_loss: 0.1689 - val_accuracy: 0.9421
Epoch 4/10
14/14 [==============================] - 21s 1s/step - loss: 0.2277 - accuracy: 0.9060 - val_loss: 0.1047 - val_accuracy: 0.9669
Epoch 5/10
14/14 [==============================] - 21s 1s/step - loss: 0.1658 - accuracy: 0.9266 - val_loss: 0.1051 - val_accuracy: 0.9669
Epoch 6/10
14/14 [==============================] - 21s 1s/step - loss: 0.1463 - accuracy: 0.9381 - val_loss: 0.0891 - val_accuracy: 0.9669
Epoch 7/10
14/14 [==============================] - 21s 1s/step - loss: 0.1643 - accuracy: 0.9335 - val_loss: 0.1117 - val_accuracy: 0.9669
Epoch 8/10
14/14 [==

In [28]:
#save the model

model.save("forest2.h5")

In [29]:
!tar -zcvf model.tgz forest2.h5

forest2.h5


In [31]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 22.3 MB/s eta 0:00:01


In [32]:
from ibm_watson_machine_learning import APIClient

API_KEY = "Ku7_PaAkU2N_uKvYMDM92hH3JaMI3cy7KvgNyHFgrYWK"

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

In [33]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [35]:
space = client.spaces.get_details()
space

{'resources': []}

In [37]:
space_uid = guid_from_space_name(client, 'Forest fire detection')
print("Space UID: ", space_uid)

Space UID:  600a9f23-c75f-4b3f-918d-ae0f32d43d12


In [38]:
client.set.default_space(space_uid)

'SUCCESS'

In [39]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [40]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [42]:
model_id

'845af93b-dbd5-482c-b67f-4524a59dbe09'

In [41]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'